<h1 style="text-align:center; font-size:200%;">FSTT Web Scraping</h1>




<h1 style="text-align:center; font-size:200%;">1.Formation FSTT</h1>



In [93]:
import requests
from bs4 import BeautifulSoup
from requests.exceptions import RequestException, ConnectionError, Timeout
import time
import json

In [18]:
def title():
    url = "https://fstt.ac.ma/Portail2023/formation-initiale/"
    retries = 3
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)
            soup = BeautifulSoup(response.content, 'html.parser')
            title_tag = soup.find('h2', class_='elementor-heading-title elementor-size-default')
            if title_tag:
                title = title_tag.text.strip()
                print(title)
                return title
            else:
                print("Title not found")
                return None
        except (ConnectionError, Timeout) as e:
            print(f"Attempt {attempt + 1}: An error occurred: {e}. Retrying...")
            time.sleep(5)  # Wait for 5 seconds before retrying
        except RequestException as e:
            print(f"Attempt {attempt + 1}: An HTTP error occurred: {e}. Retrying...")
            time.sleep(5)
    print("Failed to retrieve the page after multiple attempts.")
    return None

In [19]:
title()

FORMATION INITIALE


'FORMATION INITIALE'

In [113]:
# Helper function to fetch and parse a page
def fetch_page(url, retries=3, timeout=10):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    for attempt in range(retries):
        try:
            response = requests.get(url, headers=headers, timeout=timeout)
            response.raise_for_status()
            return BeautifulSoup(response.content, 'html.parser')
        except (ConnectionError, Timeout) as e:
            print(f"Attempt {attempt + 1}: An error occurred: {e}. Retrying...")
            time.sleep(5)
        except RequestException as e:
            print(f"Attempt {attempt + 1}: An HTTP error occurred: {e}. Retrying...")
            time.sleep(5)
    print("Failed to retrieve the page after multiple attempts.")
    return None

In [26]:
def get_deust_formations():
    url = "https://fstt.ac.ma/Portail2023/deust/"
    soup = fetch_page(url)
    if not soup:
        return []
    deust_formations = soup.find_all("h4", {'class': 'elementor-heading-title elementor-size-default'})
    return [formation.text.strip() for formation in deust_formations]


In [27]:
def get_licence_formations():
    url = "https://fstt.ac.ma/Portail2023/cycle-licence/"
    soup = fetch_page(url)
    if not soup:
        return []
    licence_formations = soup.find_all("h4", {'class': 'elementor-heading-title elementor-size-default'})
    return [formation.text.strip() for formation in licence_formations]


In [28]:
def get_master_formations():
    url = "https://fstt.ac.ma/Portail2023/cycle-master/"
    soup = fetch_page(url)
    if not soup:
        return []
    master_formations = soup.find_all("span", {'class': 'elementor-icon-list-text'})
    filtered_formations = [formation.text.strip()[len("MST : "):] for formation in master_formations if formation.text.strip().startswith("MST : ")]
    return filtered_formations

In [29]:
def get_cycle_formations():
    url = "https://fstt.ac.ma/Portail2023/cycle-ingenieur/"
    soup = fetch_page(url)
    if not soup:
        return []
    cycle_formations = soup.find_all("span", {'class': 'elementor-icon-list-text'})
    filtered_formations = [formation.text.strip() for formation in cycle_formations if formation.text.strip().startswith("DI: ") or "Logiciels et Systèmes Intelligents" in formation.text.strip()]
    return filtered_formations

In [32]:
# Obtenir les formations de chaque type
deust_formations = get_deust_formations()
licence_formations = get_licence_formations()
master_formations = get_master_formations()
cycle_formations = get_cycle_formations()

In [35]:
 # Afficher les résultats
print("Formations DEUST:\n")
for formation in deust_formations:
    print(formation)

print("\nFormations Licence:\n")
for formation in licence_formations:
    print(formation)

print("\nFormations Master:\n")
for formation in master_formations:
    print(formation)

print("\nFormations Cycle:\n")
for formation in cycle_formations:
    print(formation)


Formations DEUST:

BIOLOGIE-CHIMIE-GEOLOGIE
GÉNIE ELECTRIQUE – GÉNIE MÉCANIQUE
MATHÉMATIQUES-INFORMATIQUE-PHYSIQUE
MATHÉMATIQUES-INFORMATIQUE-PHYSIQUE-CHIMIE

Formations Licence:

Analytique des données
Biotechnologies (Options : animale et végétale)
Design Industriel et Productique (DIP)
Énergies Renouvelables (EnR)
Génie civil
Génie des Procédés
Génie Électrique Option: Génie
Électrique & Système Industriel
Génie Industriel
Génie Informatique
Géosciences Appliquées
Ingénierie de Développement d’Applications
Informatiques
Ingénierie Statistique
Mathématiques et Applications
Mathématiques et Informatique Décisionnelles
Risques et Ressources Naturels
Statistique et Science des Données
Techniques d'Analyses Chimiques (TAC)

Formations Master:

Analyse Appliquée et Ingénierie Statistique
Bases Cellulaires et Moléculaires en Biotechnologie
Environnement, Aquaculture et Développement Durable
Géoressources Energétiques et Réservoirs
Génie Civil
Génie des Matériaux pour Plasturgie et Métallur

In [34]:
if __name__ == "__main__":
    # Get the data
    initial_title = title()
    deust_formations = get_deust_formations()
    licence_formations = get_licence_formations()
    master_formations = get_master_formations()
    cycle_formations = get_cycle_formations()

    # Write data to FSTT_DATA.txt
    with open("FSTT_DATA.txt", "w", encoding="utf-8") as file:
        if initial_title:
            file.write(f"Title:\n{initial_title}\n\n")

        file.write("Formations DEUST:\n")
        for formation in deust_formations:
            file.write(f"- {formation}\n")
        file.write("\n")

        file.write("Formations Licence:\n")
        for formation in licence_formations:
            file.write(f"- {formation}\n")
        file.write("\n")

        file.write("Formations Master:\n")
        for formation in master_formations:
            file.write(f"- {formation}\n")
        file.write("\n")

        file.write("Formations Cycle Ingénieur:\n")
        for formation in cycle_formations:
            file.write(f"- {formation}\n")
        file.write("\n")

FORMATION INITIALE





<h1 style="text-align:center; font-size:200%;">Save FORMATION INITIALE as json file</h1>



In [118]:
initial_title = title()
deust_formations = get_deust_formations()
licence_formations = get_licence_formations()
master_formations = get_master_formations()
cycle_formations = get_cycle_formations()

# Create a dictionary to hold the data
data = {
    "Title": initial_title,
    "Formations": {
        "Formations DEUST": deust_formations,
        "Formations Licence": licence_formations,
        "Formations Master": master_formations,
        "Formations Cycle Ingénieur": cycle_formations
    }
}

# Write data to FSTT_DATA.json
with open("Formation_initial.json", "w", encoding="utf-8") as file:
    json.dump(data, file, ensure_ascii=False, indent=4)




<h1 style="text-align:center; font-size:200%;">2.Formation DEUST</h1>



In [49]:
def DEUST(nom_formation, file=None):
    # Construct the URL based on the provided formation name
    url = f"https://fstt.ac.ma/Portail2023/{nom_formation}/"
    
    # Fetch and parse the page
    soup = fetch_page(url)
    if not soup:
        print(f"Failed to retrieve the page for {nom_formation}.")
        return

    # Prepare the output string
    output = f"\nInformation for {nom_formation}:\n"
    
    # Find the active div containing course information
    active_div = soup.find('div', class_='eael-tabs-content')
    
    # Print course information if found
    if active_div:
        paragraphs = active_div.find_all('p', align='justify')
        if paragraphs:
            for paragraph in paragraphs:
                output += paragraph.text.strip() + '\n'
    else:
        output += "No course information found.\n"
    
    # Find the table with id 'Mpro'
    table = soup.find('table', id='Mpro')
    
    # Print semester-wise course details if found
    if table:
        semesters = table.find_all('tr')
        for semester in semesters:
            semester_name = semester.find('th').text.strip()
            courses = semester.find('td').text.strip()
            courses = courses.replace('• ', '\n- ')
            output += f"\n{semester_name}:\n"
            output += courses + '\n'
    else:
        output += "No semester-wise course details found.\n"
    
    # Find the coordinator information
    coordinator_div = soup.find('div', id='coordinateur-tab')
    
    # Print coordinator information if found
    if coordinator_div:
        coordinator_info = coordinator_div.text.strip()
        coordinator_name, contact_info = coordinator_info.split(':', 1)
        result = f"{coordinator_name.strip()} :{contact_info.strip()}"
        output += f"\n{result}\n"
    else:
        output += "Coordinator information not found.\n"
    
    # If file is provided, write to it; otherwise, print to console
    if file:
        file.write(output)
    else:
        print(output)


In [50]:
DEUST('biologie-chimie-geologie')


Information for biologie-chimie-geologie:
Le tronc commun BCG est un cursus qui comporte un ensemble de modules pris dans plusieurs champs disciplinaires et ayant pour objectif de faire acquérir des connaissances, des aptitudes et des compétences. Donner à l’étudiant les outils méthodologique pour qu’il puisse organiser son travail et développer un esprit de synthèse, de rigueur et d’initiative.
L’objectif de cette formation et de donner à l’étudiant les possibilités d'orienter sa carrière vers les licences de chimie, de biologie et de géologie.
Également, ce tronc commun permet à l’étudiant de postuler à des concours d’accès aux cycles ingénieurs de notre établissement ainsi qu’au concours nationaux ou internationaux des écoles d’ingénieurs.

Semestre 1:

- Biologie cellulaire
- Optique et Radioactivité
- Cosmologie & Géodynamique interne
- Structure de la matière
- Algèbre
- Langues et Communication -LC1

Semestre 2:

- Biologie animale
- Thermodynamique/ Mécanique des fluides
- Géo

In [51]:
DEUST('genie-electrique-genie-mecanique')


Information for genie-electrique-genie-mecanique:
L’objectif du tronc commun GE/GM est de donner à l’étudiant une base solide en matières scientifiques (physique, chimie et mathématiques etc…) et techniques (Electrotechnique, Automatique, mécanique et instrumentation et métrologie) qui vont lui permet de continuer ses études en licences qui émanent de ce parcours tout en gardant la possibilité de se réorienter à d’autres licences des autres troncs communs comme MIP et MIPC.
Également, ce tronc commun permet à l’étudiant de postuler à des concours nationaux ou internationaux des écoles d’ingénieurs.

Semestre 1:

- Circuits électriques et électroniques
- Electricité
- Analyse 1 : Fonction d’une variable réelle
- Algèbre 1 : Polynômes et espaces vectoriels
- Algorithmique et Programmation 1
- Langues et Communication -LC1

Semestre 2:

- Thermodynamique
- Mécanique du point et Optique géométrique
- Analyse 2 : Calcul intégral et équations différentielles
- Algèbre 2 : Réduction des endo

In [52]:
DEUST('mathematique-informatique-physique')


Information for mathematique-informatique-physique:
L’objectif du tronc commun MIP est de donner à l’étudiant une base solide dans les matières scientifiques (physique, chimie et mathématiques etc…) qui vont lui permettre de continuer ses études dans les semestres S5 et S6 des cycles licences qui émanent de ce parcours tout en gardant la possibilité de se réorienter vers d’autres troncs communs comme MIPC et GE/GM.
Également, ce tronc commun permet à l’étudiant de postuler à des concours d’accès aux cycles ingénieurs dispensés dans notre établissement et aussi à des concours nationaux ou internationaux des écoles d’ingénieurs.

Semestre 1:

- Circuits électriques et électroniques
- Electricité
- Analyse 1 : Fonction d’une variable réelle
- Algèbre 1 : Polynômes et espaces vectoriels
- Algorithmique et Programmation 1
- Langues et Communication -LC1

Semestre 2:

- Thermodynamique
- Mécanique du point et Optique géométrique
- Analyse 2 : Calcul intégral et équations différentielles
- A

In [53]:
DEUST('mathematiques-informatique-physique-chimie')


Information for mathematiques-informatique-physique-chimie:
L’objectif du tronc commun MIPC est de donner à l’étudiant une base solide dans les matières scientifiques (physique, chimie et mathématiques etc…) qui vont lui permettre de continuer ses études dans les semestres S5 et S6 des cycles licences qui émanent de ce parcours tout en gardant la possibilité de se réorienter vers d’autres troncs communs comme MIP et GE/GM.
Également, ce tronc commun permet à l’étudiant de postuler à des concours d’accès aux cycles ingénieurs dispensés dans notre établissement et aussi à des concours nationaux ou internationaux des écoles d’ingénieurs.

Semestre 1:

- Circuits électriques et électroniques
- Electricité
- Analyse 1 : Fonction d’une variable réelle
- Algèbre 1 : Polynômes et espaces vectoriels
- Algorithmique et Programmation 1
- Langues et Communication -LC1

Semestre 2:

- Thermodynamique
- Mécanique du point et Optique géométrique
- Analyse 2 : Calcul intégral et équations différentie

In [54]:
# Append DEUST specific formations to the file
with open("FSTT_DATA.txt", "a", encoding="utf-8") as file:
    DEUST('biologie-chimie-geologie', file)
    DEUST('genie-electrique-genie-mecanique', file)
    DEUST('mathematique-informatique-physique', file)
    DEUST('mathematiques-informatique-physique-chimie', file)




<h1 style="text-align:center; font-size:200%;">Save DEUST Formation to json file</h1>



In [97]:
import json
def DEUST(nom_formation, file=None):
    # Construct the URL based on the provided formation name
    url = f"https://fstt.ac.ma/Portail2023/{nom_formation}/"
    
    # Fetch and parse the page
    soup = fetch_page(url)
    if not soup:
        print(f"Failed to retrieve the page for {nom_formation}.")
        return

    # Initialize the dictionary to hold the data
    formation_data = {
        "formation": {
            "Description": "",
            "Semestres": {}
        }
    }
    
    # Find the active div containing course information
    active_div = soup.find('div', class_='eael-tabs-content')
    
    # Set formation description if found
    if active_div:
        description_paragraphs = active_div.find_all('p', align='justify')
        if description_paragraphs:
            formation_data["formation"]["Description"] = "\n".join(paragraph.text.strip() for paragraph in description_paragraphs)

    # Find the table with id 'Mpro'
    table = soup.find('table', id='Mpro')
    
    # Add semester-wise course details if found
    if table:
        semesters = table.find_all('tr')
        for semester in semesters:
            semester_name = semester.find('th').text.strip()
            courses = semester.find('td').text.strip().replace('• ', '').split('\n')
            formation_data["formation"]["Semestres"][semester_name] = {"modules": courses}

    # Find the coordinator information
    coordinator_div = soup.find('div', id='coordinateur-tab')
    
    # Add coordinator information if found
    if coordinator_div:
        coordinator_info = coordinator_div.text.strip().split(':')
        coordinator_name = coordinator_info[0].strip()
        coordinator_email = coordinator_info[1].strip()
        formation_data["formation"]["Coordinator"] = coordinator_name
        formation_data["formation"]["Coordinator Email"] = coordinator_email

    # If file is provided, write to it; otherwise, print to console
    if file:
        json.dump(formation_data, file, ensure_ascii=False, indent=4)
    else:
        print(json.dumps(formation_data, ensure_ascii=False, indent=4))

# Append DEUST specific formations to the file
with open("FSTT_DATA.json", "a", encoding="utf-8") as file:
    DEUST('biologie-chimie-geologie', file)
    DEUST('genie-electrique-genie-mecanique', file)
    DEUST('mathematique-informatique-physique', file)
    DEUST('mathematiques-informatique-physique-chimie', file)





<h1 style="text-align:center; font-size:200%;">3.Formation Licence</h1>



In [55]:
def Licence(nom_formation, file=None):
    # Construct the URL based on the provided formation name
    url = f"https://fstt.ac.ma/Portail2023/{nom_formation}/"
    
    # Fetch and parse the page
    soup = fetch_page(url)
    if not soup:
        print(f"Failed to retrieve the page for {nom_formation}.")
        return

    # Prepare the output string
    output = f"\nInformation for {nom_formation}:\n"
    
    # Find the active div containing course information
    active_div = soup.find('div', class_='eael-tabs-content')
    
    # Print course information if found
    if active_div:
        paragraphs = active_div.find_all('p', align='justify')
        if paragraphs:
            for paragraph in paragraphs:
                output += paragraph.text.strip() + '\n'
    else:
        output += "No course information found.\n"
                
    # Find the competences
    competences_div = soup.find('div', id='competences-visees-et-debouches-tab')
    
    # Print competences if found
    if competences_div:
        # Option 1: Check for ul tag
        competences_list = competences_div.find('ul')
        if competences_list:
            competences_items = competences_list.find_all('li')
            if competences_items:
                output += "\nCompetences:\n"
                for competence in competences_items:
                    output += competence.text.strip() + '\n'
        
        # Option 2: Check for p tag
        else:
            competences_paragraphs = competences_div.find_all('p', align='justify')
            if competences_paragraphs:
                output += "\nCompetences:\n"
                for paragraph in competences_paragraphs:
                    output += paragraph.text.strip() + '\n'
        
    else:
        output += "Competences not found.\n"
        
    # Find the table with id 'Mpro'
    table = soup.find('table', id='Mpro')
    
    # Print semester-wise course details if found
    if table:
        semesters = table.find_all('tr')
        for semester in semesters:
            semester_name = semester.find('th').text.strip()
            courses = semester.find('td').text.strip()
            courses = courses.replace('• ', '\n- ')
            output += f"\n{semester_name}:\n"
            output += courses + '\n'
    else:
        output += "No semester-wise course details found.\n"
    
    # Find the coordinator information
    coordinator_div = soup.find('div', id='coordinateur-tab')
    
    # Print coordinator information if found
    if coordinator_div:
        coordinator_info = coordinator_div.text.strip()
        coordinator_name, contact_info = coordinator_info.split(':', 1)
        result = f"{coordinator_name.strip()} :{contact_info.strip()}"
        output += f"\n{result}\n"
    else:
        output += "Coordinator information not found.\n"
    
    # If file is provided, write to it; otherwise, print to console
    if file:
        file.write(output)
    else:
        print(output)

In [57]:
Licence('lst-analytique-des-donnees')


Information for lst-analytique-des-donnees:
La Licence Science et Techniques en analytique des données permet aux étudiants de doter de compétences en matière d'outils informatiques, des
techniques et des méthodes statistiques pour permettre d’organiser, de synthétiser et de traduire efficacement les données métier d’une
organisation. L'étudiant doit être en mesure d'apporter un appui analytique à la conduite d'exploration et à l'analyse complexe de données.

Competences:
Masters en sciences de données: fouille de données, business analytiques, blockchain,
Masters orientés e-Technologies: e-Business, e-Administration et e-Logistique
Formations d’Ingénieurs dans une école d’ingénieurs à l’issue de la deuxième ou de la troisième année de licence
Data scientist
Technicien supérieur en SGBD R : installation, configuration et administration des SGBD
WebMaster et développeur de sites web dynamiques
Intégration du monde du travail dans les entreprises et les bureaux d’études

Semestre 5:

- 

In [58]:
Licence('lst-biotechnologies')


Information for lst-biotechnologies:
L’objectif de la licence Biotechnologie consiste à former des étudiants dans les différents champs disciplinaires de la biologie moderne et de permettre aux étudiants d’acquérir de solides connaissances scientifiques pour une réelle compréhension du Vivant. L’enseignement est conçu pour allier concepts fondamentaux et apprentissage des outils méthodologiques. Il est également conçu pour étendre les compétences au-delà du domaine de la biologie, en particulier vers le monde économique. Ce parcours de licence vise donc à :

Competences:
Les métiers de l'enseignement primaire, secondaire.
Les métiers de la recherche appliquée et du développement dans les secteurs des biotechnologies, de l'agro-  alimentaire, de l'industrie pharmaceutique,...
Les métiers de la vente (délégués médicaux, technico-commerciaux, entretien d'appareillages scientifiques...).
Les métiers du conseil (consultants, experts auprès de cabinets juridiques ou d'institutions).
Les mét

In [59]:
Licence('lst-design-industriel-et-productique')


Information for lst-design-industriel-et-productique:
Le Lauréat de Licence en Design et Productique en Génie Mécanique, à travers sa formation technique et scientifique, sera en mesure de suivre et d’intervenir dans toutes les étapes de Design, de réalisation, de Contrôle et de suivi du produit tout au long de son cycle de vie.

Competences:
Planification et ordonnancement des opérations de fabrication ;
Participation à l’élaboration d’un cahier de charge de produit mécanique ;
Gestion de la production mécanique ;
Maintenance des équipements mécaniques (Roulements, embrayages, pompes, …) ;
Contrôle de la conformité des produits dans les Laboratoires et Département qualité : Spécification géométriques
        et d’état de surface, Essais Mécaniques, Traitements thermiques des matériaux ;
Intégration des Services Techniques d’Achat de vente et après-vente.

Semestre 5:

- Choix des matériaux, dimensionnement et essais
- Eléments de machines
- Machines industrielles
- Management Industr

In [60]:
Licence('lst-genie-civil')


Information for lst-genie-civil:
La licence Génie Civil conduit vers un Master ou une formation d’ingénieur. Elle permet également                   une insertion professionnelle, grâce aux différentes compétences acquises durant ce parcours.
Cette insertion peut être dans des secteurs d’activités, comme des bureaux d’études ‘en Génie Civil, en environnement, en aménagement… etc), des entreprises du secteur du bâtiment et des travaux publics, des cabinets de maîtrise d’œuvre, des services de maîtrise d’œuvre, des services d’ouvrages ou de bureaux de contrôle technique.

Competences:
La licence Génie Civil conduit vers un Master ou une formation d’ingénieur. Elle permet également                   une insertion professionnelle, grâce aux différentes compétences acquises durant ce parcours.
Cette insertion peut être dans des secteurs d’activités, comme des bureaux d’études ‘en Génie Civil, en environnement, en aménagement… etc), des entreprises du secteur du bâtiment et des travaux publ

In [61]:
Licence('lst-energies-renouvelables')


Information for lst-energies-renouvelables:
Approfondir les connaissances scientifiques et techniques en énergies renouvelables. Ce domaine inclut l’énergie solaire, éolienne, géothermique, hydraulique, thermoélectrique, L’énergie de la biomasse, le stockage de l’énergie, l’efficacité énergétique.
Elle permettra aux étudiants de pouvoir créer leur entreprise pour l’installation et l’entretien des mini installations photovoltaïques, éolienne, donner du conseil au niveau EnR, ou intégrer des cycles d’ingénieurs ou de Master spécialisés.
Lors de cette formation, les étudiants auront à étudier les différentes formes de production des énergies renouvelables (EnR). Ils apprendront les fondements scientifiques et techniques sur l’énergie solaire (photovoltaïque, thermique), éolien, hydraulique, géothermique, l’énergie de biomasse et déchets, le stockage d’énergie, et l’efficacité énergétique.

Competences:
Lors de cette formation, les étudiants auront à étudier les différentes formes de prod

In [62]:
Licence('lst-8-genie-des-procedes')


Information for lst-8-genie-des-procedes:
Formation visant à donner aux étudiants des bases solides en chimie, physique, mathématiques, informatique et de gestion des procédés industriels. Les lauréats de la formation seront capables de conduire des ateliers de fabrication continue et séquentielle, de manager une équipe, de connaître et de prendre en compte les contraintes de sécurité, d’environnement et de l’organisation scientifique du travail.

Competences:
Mettre l’étudiant au cœur des problèmes technologique réel qu’il devra résoudre.
Le futur lauréat doit convaincre le secteur industriel et laboratoires de ses capacités scientifiques, multidisciplinaires, décisionnelles et organisationnelles.

Semestre 5:

- Calculs des réacteurs
- Bilan Matière et Énergie
- Opérations Unitaires
- Mécanique des fluides et Hydraulique
- Optimisation des Procédés Industrielle
- Modélisation des processus et Régulation des Procédés

Semestre 6:

- Matériaux et Industrie chimique
- Procédés de Dépol

In [63]:
Licence('lst-genie-electrique-option-genieelectrique-systeme-industriel')


Information for lst-genie-electrique-option-genieelectrique-systeme-industriel:
L’objectif de la licence Génie électrique est de donner aux étudiants les éléments de base en physique mathématique et informatique et de leur apporter une formation solide dans les domaines du génie électrique, en particulier en électronique électrotechnique. Ce qui leur permettra de préparer un master à dominante ingénierie EEA ou d’intégrer les grandes écoles d’ingénieurs. Ils pourront aussi se présenter aux différentes fonctions publiques ou privées exigeant le niveau de la licence.
Les étudiants sont également initiés à la vie professionnelle par le biais de stages au sein des entreprises.

Competences:
Électronique de contrôle, de mesure et de commande.
Traitement et analyse du signal.
Principe de fonctionnement et le domaine d’application des machines électriques.
Électronique de puissance.
Automates programmables.
Techniques de commande de processus industriels.
Systèmes à microprocesseur.
Réseaux 

In [64]:
Licence('lst-genie-industriel')


Information for lst-genie-industriel:
Cette formation pluridisciplinaire garantit l’ouverture d’esprit nécessaire pour appréhender et dominer les problèmes concernant un projet industriel dont l’aspect technique ne constitue qu’une partie parmi d’autres.
L’étudiant disposant d’une licence Sciences et techniques Génie Industriel pourra poursuivre ses études supérieures dans les différentes grandes écoles ou bien en Master Science et Techniques. Il pourra aussi intégrer le tissu industriel essentiellement les PME/PMI.

Competences:
Métiers de l’ingénieur : Production ; Méthodes ; Maintenance ; Management de la qualité.
Poursuite des études en Master ou en cycle d’ingénieur.

Semestre 5:

- Machines Hydrauliques
- Gestion de production
- Machines Thermiques
- Gestion de la qualité
- Gestion de la maintenance et sureté de fonctionnement
- Matériaux et RDM

Semestre 6:

- Logistique et Supply Chain Management
- Optimisation des Systèmes
- Management de projet
- Projet de fin d’études

Coor

In [65]:
Licence('lst-genie-informatique')


Information for lst-genie-informatique:

Technicien supérieur en développement d’application en C++ et JAVA ; Technicien supérieur en réseaux locaux ; Technicien supérieur en SGBD-R : installation, configuration et administration des SGBD ; WebMaster et développeur des sites web dynamiques, ...
Les orientations potentielles de poursuite des études sont de type : Masters à dominance informatique, Masters orientés e-Technologies, Masters à dominance automatique ou informatique industrielle ou informatique décisionnelle et Formations d’ingénieurs.

Competences:
Technicien supérieur en développement d’application en C++ et JAVA ; Technicien supérieur en réseaux locaux ; Technicien supérieur en SGBD-R : installation, configuration et administration des SGBD ; WebMaster et développeur des sites web dynamiques, ...
Les orientations potentielles de poursuite des études sont de type : Masters à dominance informatique, Masters orientés e-Technologies, Masters à dominance automatique ou informat

In [66]:
Licence('lst-geosciences-appliquees')


Information for lst-geosciences-appliquees:

Competences:
L’expertise et Cartographie.
La Géotechnique et Géo-matériaux (achat et vente du matériel de génie géologique).
L’environnement (technicien de l’environnement).

Semestre 5:

- Pétrographie/Minéralogie
- Environnements sédimentaires
- Élément de  tectonique/Domaines structuraux du Maroc
- Ressources hydriques
- Géochimie et pollution urbaine
- Pédologie

Semestre 6:

- Traitement des données géologiques
- Cartographie numérique
- Géologie appliquée
- Projet de fin d’études

Coordinnateur pédagogique :Pr.Wahbi Miriam           :  mwahbi@uae.ac.ma


In [67]:
Licence('lst-ingenierie-de-developpement-dapplications-informatiques')


Information for lst-ingenierie-de-developpement-dapplications-informatiques:

Competences:
Technicien supérieur en développement d’application en C++ et JAVA
Technicien supérieur en réseaux locaux : installation, configuration et administration des réseaux locaux
Technicien supérieur en SGBD R : installation, configuration et administration des SGBD
WebMaster et développeur de sites web dynamiques
Poursuivre des études supérieures en Informatique

Semestre 5:

- Modélisations avancée et Méthodes de génie logiciel
- Développement Web
- Base de données Structurées et non Structurées
- Programmation Orientée Objet en C++/Java
- Systèmes et réseaux informatiques
- Développement de soft skills

Semestre 6:

- Innover, Entreprendre et s’initier à la Gestion d'une Entreprise avec un ERP
- Initiation en développement mobile et Edge Computing
- Développement web avancé Back end (Python) 
- PFE

Coordinnateur :Pr.KOUNAIDI Mohamed             :  m.kounaidi@@uae.ac.ma


In [68]:
Licence('lst-ingenierie-statistique')


Information for lst-ingenierie-statistique:

Competences:
Formation académique en mathématiques fondamentales et statistiques mathématiques offrant ainsi la possibilité de poursuivre les études en Master fondamental ou professionnel.
Maîtrise de l’outil informatique et les logiciels statistiques spécialisés.
Capacité à concevoir et à mener des études statistiques offrant des perspectives professionnelles dans différents secteurs socio-économiques et industriels.

Semestre 5:

- Topologies dans les espaces métriques
- Intégration
- Probabilités avancées
- Recherche Opérationnelle
- Concept de base de Programmation avancée C++
- Analyse des données

Semestre 6:

- Statistique Mathématique
- MSP
- Régression–Anova
- Projet de fin d’études

Coordinnateur pédagogique :Pr.El Merzguioui Mhamed             :  melmerzguioui@uae.ac.ma


In [69]:
Licence('lst-mathematiques-et-applications')


Information for lst-mathematiques-et-applications:
Acquérir une double compétence mathématiques et informatique grâce à laquelle ils aborderont les problèmes de modélisation et de simulation scientifique sous tous leurs aspects, de la conception à la mise en oeuvre numérique.
L’accent est donc mis à la fois sur des techniques d’ingénierie mathématiques et sur les langages de programmation et les outils algorithmiques.

Competences:
Les entreprises et les bureaux d’études ;
La poursuite des études en cycle ingénieur ;
La poursuite des études en cycle Master ;
L’enseignement public ou privé.

Semestre 5:

- Topologies dans les espaces métriques
- Intégration
- Probabilités avancées
- Recherche Opérationnelle
- Concepts de base de programmation orientée C
- Analyse numérique matricielle

Semestre 6:

- Calcul différentiel et équations différentielles
- Topologie générale et distributions
- Analyse numérique des équations différentielles
- Projet de fin d’études

Coordinnateur pédagogique

In [70]:
Licence('lst-mathematiques-et-informatique-decisionnelles')


Information for lst-mathematiques-et-informatique-decisionnelles:

Competences:
Master de mathématiques et informatique (fondamental, spécialisé ou professionnel) dans une université marocaine ou étrangère.
Intégration (sur concours ou dossier) des écoles d'ingénieurs
Administration
Industries et commerce
Sociétés financières.
Bureaux d’études

Semestre 5:

- Base de données Relationnelles et Nosql
- POO/ Développement web
- Intégration et Probabilités
- Topologie et Calcul différentiel
- Optimisation non linéaire et recherche opérationnelle
- Anglais et Management de Projet

Semestre 6:

- PYTHON ET MACHINE LEARNING
- Mathématiques pour l'Intelligence Artificiel
- Analyse numérique et équations différentielles 
- PFE

Coordinnateur :Pr.LAHROUZ Adil             :  a.lahrouz@uae.ac.ma


In [71]:
Licence('lst-rrn')


Information for lst-rrn:

Competences:
Observatoire National des Risques Naturels,
Bureaux d’étude,
Laboratoire d’étude (LPEE),
Organismes gestionnaires des ressources hydriques, de sols, minières, ,
Les institutions administratives et professionnelles,
Les institutions de recherche scientifique et technique,
Etc.

Semestre 5:

- Risques et ressources naturels
- Techniques géophysiques/géotechniques/géochimiques
- Techniques géomatiques
- Informatiques
- Modélisation et analyse de données
- Soft Skills

Semestre 6 ( Option : Risques naturels):

- Risques géologiques  
- Risques hydroclimatiques
- Risques naturels et aménagement
- Projet de fin d’études

Semestre 6 ( Option : Ressources naturelles):

- Géoressources Naturelles
- Bioressources Naturelles
- Valorisation et durabilité des ressources
- Projet de fin d’études

Coordinnateur :Pr.Bouzid Saida              : sbouzid@uae.ac.ma


In [72]:
Licence('lst-statistique-et-science-des-donnees')


Information for lst-statistique-et-science-des-donnees:

Competences:
Master de mathématiques et informatique (fondamental, spécialisé ou professionnel) dans une université marocaine ou étrangère.
Intégration (sur concours ou dossier) des écoles d'ingénieurs
Administration
Industries et commerce
Sociétés financières.
Bureaux d’études

Semestre 5:

- Base de données Relationnelles et NoSQL
- Programmation orientée objet et Développement Web
- Statistique Inférentielle et Recherche Opérationnelle
- Intégration et Probabilités
- Analyse des Données et Datamining
- Anglais et Management de Projet

Semestre 6:

- Programmation Python et Machine Learning
- Mathématiques pour l’Intelligence Artificielle
- Régression – Anova et Maîtrise Statistique des Procédés 
- PFE

Coordinnateur :Pr.EL MERZGUIOUI Mhamed             :  melmerzguioui@uae.ac.ma


In [73]:
Licence('lst-techniques-danalyses-chimiques')


Information for lst-techniques-danalyses-chimiques:

Semestre 5:

- Chimie organique avancée
- Chimie inorganique
- Thermochimie, Cinétique et Catalyse
- Electrochimie et Méthodes électro-analytiques
- Méthodes Spectroscopiques
- Techniques d’analyse inorganiques

Semestre 6:

- Techniques Séparatives
- Chimie et analyse des eaux
- Assurance qualité dans les laboratoires d’analyses chimiques
- Projet de fin d’études

Coordinnateur pédagogique :Pr.Chabbi Mohamed            :  Jbilou Mohammed


In [76]:
with open("FSTT_DATA.txt", "a", encoding="utf-8") as file:
    Licence('lst-analytique-des-donnees', file)
    Licence('lst-biotechnologies', file)
    Licence('lst-design-industriel-et-productique', file)
    Licence('lst-energies-renouvelables', file)
    Licence('lst-genie-civil', file)
    Licence('lst-8-genie-des-procedes', file)
    Licence('lst-genie-electrique-option-genieelectrique-systeme-industriel', file)
    Licence('lst-genie-industriel', file)
    Licence('lst-genie-informatique', file)
    Licence('lst-geosciences-appliquees', file)
    Licence('lst-ingenierie-de-developpement-dapplications-informatiques', file)
    Licence('lst-ingenierie-statistique', file)
    Licence('lst-mathematiques-et-applications', file)
    Licence('lst-mathematiques-et-informatique-decisionnelles', file)
    Licence('lst-rrn', file)
    Licence('lst-statistique-et-science-des-donnees', file)
    Licence('lst-techniques-danalyses-chimiques', file)




<h1 style="text-align:center; font-size:200%;">Save Licence as json</h1>



In [109]:
import json
def Licence(nom_formation, file="Formation_licence.json"):
    # Construct the URL based on the provided formation name
    url = f"https://fstt.ac.ma/Portail2023/{nom_formation}/"
    
    # Fetch and parse the page
    soup = fetch_page(url)
    if not soup:
        print(f"Failed to retrieve the page for {nom_formation}.")
        return

    # Initialize dictionary to store information
    data = {}
    
    # Add formation name to dictionary
    data[nom_formation] = {}

    # Find the description div
    description_div = soup.find('div', class_='description')
    if description_div:
        data[nom_formation]['Description'] = description_div.text.strip()
    else:
        data[nom_formation]['Description'] = "Description not found."
    
    # Find the competences
    competences_div = soup.find('div', id='competences-visees-et-debouches-tab')
    if competences_div:
        competences_list = competences_div.find('ul')
        if competences_list:
            competences_items = competences_list.find_all('li')
            if competences_items:
                data[nom_formation]['Competences'] = [competence.text.strip() for competence in competences_items]
        else:
            competences_paragraphs = competences_div.find_all('p', align='justify')
            if competences_paragraphs:
                data[nom_formation]['Competences'] = [paragraph.text.strip() for paragraph in competences_paragraphs]
    else:
        data[nom_formation]['Competences'] = ["Competences not found."]
    
    # Find the table with id 'Mpro'
    table = soup.find('table', id='Mpro')
    
    # Parse semester-wise course details
    if table:
        semesters = table.find_all('tr')
        for semester in semesters:
            semester_name = semester.find('th').text.strip()
            courses = semester.find('td').text.strip()
            courses_list = courses.split('• ')
            data[nom_formation][semester_name] = [course.strip() for course in courses_list if course.strip()]
    else:
        data[nom_formation]['Semesters'] = "No semester-wise course details found."
    
    # Find the coordinator information
    coordinator_div = soup.find('div', id='coordinateur-tab')
    
    # Parse coordinator information
    if coordinator_div:
        coordinator_info = coordinator_div.text.strip()
        coordinator_name, contact_info = coordinator_info.split(':', 1)
        data[nom_formation]['Coordinnateur'] = coordinator_name.strip()
        data[nom_formation]['Coordinnateur Email'] = contact_info.strip()
    else:
        data[nom_formation]['Coordinnateur'] = "Coordinator information not found."
        data[nom_formation]['Coordinnateur Email'] = ""

    # Read existing data from JSON file
    try:
        with open(file, 'r', encoding='utf-8') as json_file:
            existing_data = json.load(json_file)
    except FileNotFoundError:
        existing_data = {}

    # Update existing data with new data
    existing_data.update(data)
    
    # Write updated data to JSON file
    with open(file, 'w', encoding='utf-8') as json_file:
        json.dump(existing_data, json_file, indent=4, ensure_ascii=False)
    
    print(f"Information for {nom_formation} saved to {file}.")

In [110]:
Licence('lst-analytique-des-donnees')
Licence('lst-biotechnologies')
Licence('lst-design-industriel-et-productique')
Licence('lst-energies-renouvelables')
Licence('lst-genie-civil')
Licence('lst-8-genie-des-procedes')
Licence('lst-genie-electrique-option-genieelectrique-systeme-industriel')
Licence('lst-genie-industriel')
Licence('lst-genie-informatique')
Licence('lst-geosciences-appliquees')
Licence('lst-ingenierie-de-developpement-dapplications-informatiques')
Licence('lst-ingenierie-statistique')
Licence('lst-mathematiques-et-applications')
Licence('lst-mathematiques-et-informatique-decisionnelles')
Licence('lst-rrn')
Licence('lst-statistique-et-science-des-donnees')
Licence('lst-techniques-danalyses-chimiques')

Information for lst-analytique-des-donnees saved to Formation_licence.json.
Information for lst-biotechnologies saved to Formation_licence.json.
Information for lst-design-industriel-et-productique saved to Formation_licence.json.
Information for lst-energies-renouvelables saved to Formation_licence.json.
Information for lst-genie-civil saved to Formation_licence.json.
Information for lst-8-genie-des-procedes saved to Formation_licence.json.
Information for lst-genie-electrique-option-genieelectrique-systeme-industriel saved to Formation_licence.json.
Information for lst-genie-industriel saved to Formation_licence.json.
Information for lst-genie-informatique saved to Formation_licence.json.
Information for lst-geosciences-appliquees saved to Formation_licence.json.
Information for lst-ingenierie-de-developpement-dapplications-informatiques saved to Formation_licence.json.
Information for lst-ingenierie-statistique saved to Formation_licence.json.
Information for lst-mathematiques-et-ap




<h1 style="text-align:center; font-size:200%;">3.Formation Master</h1>



In [77]:
def Master(nom_formation, file=None):
    # Construct the URL based on the provided formation name
    url = f"https://fstt.ac.ma/Portail2023/{nom_formation}/"
    
    # Fetch and parse the page
    soup = fetch_page(url)
    if not soup:
        print(f"Failed to retrieve the page for {nom_formation}.")
        return

    # Prepare the output string
    output = f"\nInformation for {nom_formation}:\n"
    
    # Find the active div containing course information
    active_div = soup.find('div', class_='eael-tabs-content')
    
    # Print course information if found
    if active_div:
        paragraphs = active_div.find_all('p', align='justify')
        if paragraphs:
            for paragraph in paragraphs:
                output += paragraph.text.strip() + '\n'
    else:
        output += "No course information found.\n"
                
    # Find the competences
    competences_div = soup.find('div', id='competences-visees-et-debouches-tab')
    
    # Print competences if found
    if competences_div:
        # Option 1: Check for ul tag
        competences_list = competences_div.find('ul')
        if competences_list:
            competences_items = competences_list.find_all('li')
            if competences_items:
                output += "\nCompetences:\n"
                for competence in competences_items:
                    output += competence.text.strip() + '\n'
        
        # Option 2: Check for p tag
        else:
            competences_paragraphs = competences_div.find_all('p', align='justify')
            if competences_paragraphs:
                output += "\nCompetences:\n"
                for paragraph in competences_paragraphs:
                    output += paragraph.text.strip() + '\n'
        
    else:
        output += "Competences not found.\n"
        
    # Find the table with id 'Mpro'
    table = soup.find('table', id='Mpro')
    
    # Print semester-wise course details if found
    if table:
        semesters = table.find_all('tr')
        for semester in semesters:
            semester_name = semester.find('th').text.strip()
            courses = semester.find('td').text.strip()
            courses = courses.replace('• ', '\n- ')
            output += f"\n{semester_name}:\n"
            output += courses + '\n'
    else:
        output += "No semester-wise course details found.\n"
    
    # Find the coordinator information
    coordinator_div = soup.find('div', id='coordinateur-tab')
    
    # Print coordinator information if found
    if coordinator_div:
        coordinator_info = coordinator_div.text.strip()
        coordinator_name, contact_info = coordinator_info.split(':', 1)
        result = f"{coordinator_name.strip()} :{contact_info.strip()}"
        output += f"\n{result}\n"
    else:
        output += "Coordinator information not found.\n"
    
    # If file is provided, write to it; otherwise, print to console
    if file:
        file.write(output)
    else:
        print(output)

In [83]:
Master('mst-analyse-appliquee-etingenierie-statistique')

Le Master Analyse Appliquée et Ingénierie Statistique (MAAIS) de l'Université Abdelmalek Essaadi a pour vocation d'offrir des parcours qui trouvent leurs bases dans une approche pluridisciplinaire. Au département de mathématiques on propose aux étudiants d’acquérir une double compétence en mathématiques et en statistiques appliquées s’appuyant sur un tronc commun développant des concepts et des outils de base. Pour une approche pluridisciplinaire des problèmes liés aux applications, les mathématiques et les statistiques appliquées ne peuvent que former des étudiants pour la recherche et les activités professionnelles de haut niveau avec un bon bagage mathématique. La spécialisation se ferait au terme du second semestre (S2) selon le choix des étudiants. En plus une bonne formation en tronc commun solide permettra de mieux optimiser les ressources humaines surtout au niveau des encadrants des PFEs (Master) et/ou des Doctorants.

Competences:

Donner une formation mathématique et statist

In [84]:
Master('mst-bases-cellulaires-etmoleculaires-en-')

Le master BCMB vise la formation des spécialistes dans le domaine des Biotechnologies capables de répondre aux exigences de la recherche fondamentale et appliquée, et de s’adapter au marché de l’emploi dans les secteurs socio-économique et industriel. Le lauréat sera doté d’une nouvelle conception à la biologie alliant théorie à la pratique. A l’issu de cette formation, le lauréat sera capable d’élaborer des protocoles, de maîtriser la manipulation, de pouvoir analyser et exploiter les résultats expérimentaux obtenus. Il sera aussi capable d’innover en apportant des solutions dans le domaine des Biotechnologies, d’animer une équipe et de gérer des projets, de rédiger mémoires et articles scientifiques, rapports et présenter des exposés.
L’objectif final est de former des chercheurs et des cadres de haut niveau, spécialistes en biotechnologies appliquées notamment dans les domaines de la sélection et l’amélioration génétique des plantes.
Les lauréats ont la possibilité de continuer leur

In [85]:
Master('mst-environnement-aquacultureet-developpement-durable')

Encore peu développée au Maroc, l’aquaculture offre des perspectives de développement très intéressantes. Le Maroc s’est donné tous les moyens pour insuffler une nouvelle dynamique dans le secteur halieutique, à travers le lancement du plan décennal 2015-2024 pour le développement de l’aquaculture continentale et le plan Halieutis qui vise une expansion importante de l’aquaculture marine. Les prévisions tablent sur une expansion importante de l’aquaculture, des industries et des services associés. C’est pour cette raison, et en vue de répondre aux besoins immédiats en métiers relatifs à la gestion et l’exploitation de l’environnement incluant les ressources aquacoles continentales et marines que se place le projet master interuniversitaire « Environnement, aquaculture et développement durable ( البيئة وتربية الأحياء المائية والتنمية المستدامة ).
Ce master consiste en une formation pratique et interdisciplinaire. Elle pourvoit à une intégration aisée dans la vie active professionnelle. 

In [86]:
Master('mst-georessources-energetiques-et-reservoirs')

L’activité accrue dans la recherche en géosciences appliquées aux prospections géoressources énergétiques et réservoirs s’intègre dans le cadre de la stratégie gouvernementale Marocaine « Stratégie Energétique Horizon 2030 » en tant qu’activité à fort impact économique pour le pays. C’est dans ce Contexte que l’Etat marocain s’est mis sur deux composantes en parallèle pour répondre à ses besoins en matière de l’énergie : Les Energies renouvelables (Solaire et Eolienne) et les Géoressources énergétiques (Gaz/Pétrole). Cette Dynamique a été accéléré par la mise en place des mesures fiscales en vue d’inciter l’exploration. Un dispositif qui a permis l’installation depuis 2012 de plus de 37 compagnies pétrolières étrangères, dont Repsol, Anadarko Petroleum, Kosmos Energy, Sound Energy, Chevron, deuxième compagnie pétrolière aux Etats-Unis, le géant pétrolier britannique British Petroleum (BP) et plusieurs autres en vue de mener des travaux d’exploration géologiques et géophysique pour la r

In [87]:
Master('mst-genie-civil')

Le programme vise la formation des cadres en Génie Civil. A l’issue des deux années de formation, le lauréat en Master de Génie Civil doit être capable d’intervenir dans la conception, le suivi et la réalisation de projets dans les secteurs du Bâtiments et des Travaux Publics.
La formation vise, également, à développer chez l’étudiant l’esprit d’initiative, de responsabilité et de gestion. Outre les compétences scientifique et technique, le programme dispense un ensemble d’activités facilitant l’insertion professionnelle de l’étudiant : les techniques d’expression, les langues, conférences sur des réalisations de chantier, les visites d’entreprises et de chantiers…etc.
Les connaissances et compétences acquises par la présente formation dans le domaine de Génie Civil peuvent être considérées à la pointe du savoir dans cette discipline.
Ce haut niveau de qualification renforcé par un certain savoir de gérer des projets appliqués et de création d’entreprise, dispose nos lauréats d’une hau

In [88]:
Master('mst-genie-des-materiaux-pour-plasturgie-et-metallurgie')

Le Master Génie des Matériaux pour Plasturgie et Métallurgie (GMPM) se positionne dans le contexte de synergie ; Formation / Recherche / Industrie, avec des enseignements en relation avec les activités des industries de la région. Le programme vise à fournir des bases de chimie, génie des matériaux, mécanique avec un lien entre les procédés d’élaboration, de synthèse et de mise en forme de la matière première ainsi que l’étude des propriétés structurales et/ou fonctionnelles des matériaux tels que les polymères, les composites, les alliages métalliques, les céramiques et les biomatériaux.

Competences:

Élaboration, caractérisation et contrôle des matériaux
Exigences liées au choix des matériaux
Maîtrise des mécanismes de dégradation des matériaux et du comportement matériau/milieu

Semestre 1:

- Cristallographie et phénomènes
- Métallurgie et matériaux métalliques
- Formulation, rhéologie et modélisation
- Verres, céramiques et matériaux
- Polymères synthétiques
- Anglais scientifiqu

In [89]:
Master('mst-genie-energetique')

Former des lauréats pour accompagner la stratégie énergétique nationale et mettre sur le marché du travail des jeunes « énergéticiens » capables de contribuer à la réalisation des différents projets dans le domaine d’énergétique.
La formation vise à donner aux lauréats des compétences scientifiques dans le domaine de l’énergétique qui leur permettront de pouvoir intervenir pour la réalisation des audits énergétiques et réaliser le bilan des divers systèmes énergétiques et ceci pour améliorer leur efficacité énergétique (systèmes industriels, bâtiments….), de contribuer et d’accompagner la réalisation et le suivi des grands projets nationaux dans le domaine de l’énergétique (centrales thermiques, projets des énergies renouvelables, ….).
La formation vise aussi à former des lauréats avec des bases scientifiques nécessaires dans le domaine de l’énergétique pour pouvoir poursuivre des travaux de recherche scientifique dans le cadre des études doctorales ou en intégrant des organismes de re

In [90]:
Master('mst-ingenierie-environnementale-changement-climatique-et-developpement-durable')

Ce Master « IE2C2D » a été préparé dans le cadre d’une convention de partenariat entre l’Université Abdelmalek Essaâdi (UAE) et le Centre de Compétences Changements Climatiques du Maroc (4C Maroc) avec le soutien du Programme des Nations Unies pour le Développement (PNUD).
Ce master a pour but de répondre à la demande extrêmement forte de former des experts professionnels multidisciplinaires en changement climatique capables d’élaborer des projets et de concevoir des stratégies pour le développement.

Cette formation fournit des bases fondamentales et appliquées indispensable pour traiter des problèmes environnementaux, et des problèmes de changement climatique, et aussi initier des stratégies de développement durable.

Les modules de cette filière sont relatifs aux études de l’Ingénierie Environnementale, Changement climatique et Développement Durable. Elle offre une formation de professionnalisation et de recherche-développement.

Ces experts seront amenés à travailler dans les métie

In [91]:
Master('mst-intelligence-artificielle-et-sciences-de-donnees')

Cette formation d'excellence offre de solides connaissances en conception de systèmes d'intelligence artificielle et mathématiques appliquées afin de couvrir l'ensemble des problématiques de traitement et d'analyse des données massives que rencontre les entreprises. Elle met l'accent sur l'articulation entre apprentissage automatique, gestion et fouille de grandes masses de données, paradigmes du Big Data, représentation des connaissances, le traitement des données et sur les méthodologies récemment développées.

Semestre 1:

- THÉORIES ET SYSTÈMES DE RAISONNEMENTS INTELLIGENTS
- MATHÉMATIQUES POUR ANALYSE DE DONNÉES
- PROGRAMMATION AVANCÉE
- BASES DE DONNÉES AVANCÉES
- MACHINE LEARNING 1
- ANGLAIS ET TECHNIQUE D’EXPRESSION

Semestre 2:

- INFRASTRUCTURE ET ARCHITECTURE DES SYSTÈMES DISTRIBUES & BIG DATA
- PLATEFORMES IOT CORE: TECHNOLOGIES, DATA ET IA
- METAHEURISTIQUES & ALGORITHMES DE RECHERCHE STOCHASTIQUE
- SMA & NLP
- DATAMING & BI
- DEVELOPPEMENT PERSONNEL ET INTELLIGENCE EMOT

In [92]:
Master('mst-mobiquite-et-big-data')

Récemment, le "Big Data" et la mobiquité sont devenus un véritable enjeu stratégique et économique. Le traitement et l’exploitation massive des Méga données est désormais une priorité stratégique pour s’engager dans l’évolution. En effet, ses applications se font sentir dans des domaines aussi variés que les sciences, le marketing, le développement durable, les transports, la santé et l’éducation, et représente un potentiel énorme de création d’emplois.
Le Master a pour but, de former des étudiants capables de répondre aux besoins de collecte, stockage, traitement et analyse des données massives, afin d’accompagner les organisations dans des processus de développement stratégique.

Competences:

Big Data Architecte
Big Data Analyst
Data Scientist
Développeur Mobile
Administrateur des bases de données

Semestre 1:

- Programmation Orientée Objet Avancée : Java Et Python
- Systèmes Embarqués Et Mobilité
- Recherche Opérationnelle Et Théorie des Graphes
- IP Mobile Et Protocoles
- Adminis

In [93]:
Master('mst-modelisation-mathematique-et-science-de-donnees')

Le Master Modélisation Mathématique et Science de Données (MMSD) de l'Université Abdelmalek Essaadi a pour vocation d'offrir des parcours qui trouvent leurs bases dans une approche pluridisciplinaire. Au département de mathématiques on propose aux étudiants d’acquérir une double compétence en Modélisation Mathématiques et en Science de Données s’appuyant sur la maîtrise de l'informatique scientifique notamment pour le calcul haute Performance, la Statistique et la Science de Données. Cela peut inclure des compétences en programmation, en visualisation de données et en utilisation de bibliothèques de modélisation.
Pour une approche pluridisciplinaire des problèmes liés aux applications, la Modélisation Mathématique et la Science de Données ne peuvent que former des étudiants pour la recherche et les activités professionnelles de haut niveau avec des compétences nécessaires pour mener un travail de recherche et/ou de développement en misant sur : l’autonomie dans l’analyse d’un problème 

In [94]:
Master('mst-sciences-agroalimentaires')

L'objectif global de cette formation du Master est de former des ressources humaines dont le pays a besoin pour un développement optimal des sciences agroalimentaires.Parmi les objectifs spécifiques :

Competences:

Conduite d’un projet R&D
Caractérisation physico-chimique, nutritionnelle, organoleptique et rhéologique des aliments
Méthodes d’analyse des données et d’interprétation des résultats
Formation générale à la communication et à la connaissance de l’environnement professionnel.

Semestre 1:

- Procédés industriels
- Techniques d’analyses des biomolécules 1
- Anglais scientifique
- Biochimie alimentaire
- Microbiologie Alimentaire
- Exploitation des Données expérimentales

Semestre 2:

- Techniques d’analyses des biomolécules 2
- Production animale/Production végétale
- Marketing et économie de filière
- Réglementation et système de management de la santé et sécurité alimentaire
- Filière de production : Transformation des produits d’origine végétale 
- Filière de production : 

In [95]:
Master('mst-sciences-delenvironnement')

Cette formation permettra aux étudiants d’acquérir une spécialisation en environnement répondant aux exigences de la recherche fondamentale et appliquée, et apportant une ouverture sur les applications de la recherche dans les secteurs socio-économiques.

Competences:

D’identifier et d’évaluer les différentes pollutions et de mettre en œuvre des technologies de traitements appropriés ;
D’acquérir les grandes techniques d’instrumentation, d’analyse et de contrôle pour la protection de l’environnement ;
De se préparer aux carrières de recherche universitaire et industrielle ;

Semestre 1:

- Procédés de séparation
- Chimie de l’environnement et rejets urbains
- Statistique et Simulation numérique
- Microbiologie de l’Environnement
- Changements climatiques
- Anglais/ Expression et communication environnementale

Semestre 2:

- Fonctionnement et préservation des écosystèmes/Étude d’impact sur l’environnement
- Ressources hydriques et risque de contamination
- Risques environnementaux nat

In [96]:
Master('mst-sciences-du-littoralapproche-pluridisciplinaire')

Ce projet de Master représente le fruit d’un travail approfondi avec un financement de l’Union Européenne (Erasmus Plus - Capacity Building) impliquant 4 universités marocaines (Abdelmelk Essaâdi, IbnTofail, Mohammed V et Chouaib Doukkali). La finalité du projet est d’accompagner par la formation, la dynamique socioéconomique que connait le Maroc. En effet, avec ses 3500 kms de côte (façades Atlantique et méditerranée) le Royaume du Maroc se fixe un positionnement de Leader en Afrique et dans l’espace méditerranéen en matière de gestion intégrée du littoral et de ses infrastructures maritimes et portuaires de grande envergure. Cette gestion intégrée prend en considération l’ensemble des secteurs porteurs de richesse et de création d’emploi, à savoir, les secteurs de la pêche, le tourisme, l’aménagement du territoire, les changements climatiques, le transport, la logistique maritime etc. L’objectif final de ce projet est de former un potentiel humain compétent et rapidement opérationnel

In [97]:
Master('mst-securite-it-et-big-data')

La transformation digitale a profondément modifie tous les aspects du fonctionnement des entreprises et des organisations d’aujourd'hui. Le volume de données que les entreprises créent, manipulent et stockent augmente. De plus la dématérialisation massive des systèmes d'Information vers le cloud, l'explosion de l'internet des objets (IoT) dans un monde tout connecté accessible en mobilité avec smartphone, tablettes, la fraude sur internet via différentes techniques d'ingénierie sociale, la perte de données sensibles suite à des malversations, l'accroissement du télétravail utilisant des réseaux de communication vulnérables ( Internet, mobiles, wifi...) par conséquent les environnements informatiques des entreprises sont plus complexes qu'ils ne l'étaient auparavant, cette complexité crée une surface d'attaque étendue qui est plus difficile à surveiller et à sécuriser. Ceci oblige les entreprises et organisations à repenser leur approche du cyber sécurité, dans un cadre réglementaire de

In [98]:
Master('mst-systemes-informatiqueset-mobiles')

L’administration des systèmes informatiques et mobiles, comme le Cloud Computing et les objets Internet interconnectés, est devenue aujourd’hui d’une grande nécessité, car ces systèmes sont utilisés dans pratiquement tous les secteurs de l’industrie. En effet, l’évolution de la télécommunication, des téléphones mobiles, des capteurs à prix réduits, et des systèmes Cloud Computing, a créé ce besoin, notamment pour mieux gérer ces infrastructures, qui font, désormais, partie de la vie du citoyen de tous les jours, par exemple dans les réseaux sociaux, les domaines bancaires, les transports, le e-commerce, etc.
Ce Master a pour objectif de former des étudiants capables de répondre aux besoins de gestion et d’administration des systèmes informatiques et mobiles, tout en ayant les compétences nécessaires dans le développement d’applications spécifiques et de la sécurité, ceci afin d’accompagner les organisations dans leurs processus d’implémentation de systèmes informatiques modernes et évo

In [78]:
with open("FSTT_DATA.txt", "a", encoding="utf-8") as file:
    Master('mst-analyse-appliquee-etingenierie-statistique', file)
    Master('mst-bases-cellulaires-etmoleculaires-en-', file)
    Master('mst-environnement-aquacultureet-developpement-durable', file)
    Master('mst-georessources-energetiques-et-reservoirs', file)
    Master('mst-genie-civil', file)
    Master('mst-genie-des-materiaux-pour-plasturgie-et-metallurgie', file)
    Master('mst-genie-energetique', file)
    Master('mst-ingenierie-environnementale-changement-climatique-et-developpement-durable', file)
    Master('mst-intelligence-artificielle-et-sciences-de-donnees', file)
    Master('mst-mobiquite-et-big-data', file)
    Master('mst-modelisation-mathematique-et-science-de-donnees', file)
    Master('mst-sciences-agroalimentaires', file)
    Master('mst-sciences-delenvironnement', file)
    Master('mst-sciences-du-littoralapproche-pluridisciplinaire', file)
    Master('mst-securite-it-et-big-data', file)
    Master('mst-systemes-informatiqueset-mobiles', file)





<h1 style="text-align:center; font-size:200%;">Save Formation Master as Json</h1>



In [115]:
def Master(nom_formation, file="Formation_master.json"):
    # Construct the URL based on the provided formation name
    url = f"https://fstt.ac.ma/Portail2023/{nom_formation}/"
    
    # Fetch and parse the page
    soup = fetch_page(url)
    if not soup:
        print(f"Failed to retrieve the page for {nom_formation}.")
        return

    # Initialize dictionary to store information
    data = {}
    
    # Add formation name to dictionary
    data[nom_formation] = {}

    # Find the description div
    description_div = soup.find('div', class_='description')
    if description_div:
        data[nom_formation]['Description'] = description_div.text.strip()
    else:
        data[nom_formation]['Description'] = "Description not found."
    
    # Find the competences
    competences_div = soup.find('div', id='competences-visees-et-debouches-tab')
    if competences_div:
        competences_list = competences_div.find('ul')
        if competences_list:
            competences_items = competences_list.find_all('li')
            if competences_items:
                data[nom_formation]['Competences'] = [competence.text.strip() for competence in competences_items]
        else:
            competences_paragraphs = competences_div.find_all('p', align='justify')
            if competences_paragraphs:
                data[nom_formation]['Competences'] = [paragraph.text.strip() for paragraph in competences_paragraphs]
    else:
        data[nom_formation]['Competences'] = ["Competences not found."]
    
    # Find the table with id 'Mpro'
    table = soup.find('table', id='Mpro')
    
    # Parse semester-wise course details
    if table:
        semesters = table.find_all('tr')
        for semester in semesters:
            semester_name = semester.find('th').text.strip()
            courses = semester.find('td').text.strip()
            courses_list = [course.strip() for course in courses.split('• ') if course.strip()]
            data[nom_formation][semester_name] = courses_list
    else:
        data[nom_formation]['Semesters'] = "No semester-wise course details found."
    
    # Find the coordinator information
    coordinator_div = soup.find('div', id='coordinateur-tab')
    
    # Parse coordinator information
    if coordinator_div:
        coordinator_info = coordinator_div.text.strip()
        coordinator_name, contact_info = coordinator_info.split(':', 1)
        data[nom_formation]['Coordinnateur'] = coordinator_name.strip()
        data[nom_formation]['Coordinnateur Email'] = contact_info.strip()
    else:
        data[nom_formation]['Coordinnateur'] = "Coordinator information not found."
        data[nom_formation]['Coordinnateur Email'] = ""

    # Read existing data from JSON file
    try:
        with open(file, 'r', encoding='utf-8') as json_file:
            existing_data = json.load(json_file)
    except FileNotFoundError:
        existing_data = {}

    # Update existing data with new data
    existing_data.update(data)
    
    # Write updated data to JSON file
    with open(file, 'w', encoding='utf-8') as json_file:
        json.dump(existing_data, json_file, indent=4, ensure_ascii=False)
    
    print(f"Information for {nom_formation} saved to {file}.")

# Example usage:
Master('mst-analyse-appliquee-etingenierie-statistique')
Master('mst-bases-cellulaires-etmoleculaires-en-')
Master('mst-environnement-aquacultureet-developpement-durable')
Master('mst-georessources-energetiques-et-reservoirs')
Master('mst-genie-civil')
Master('mst-genie-des-materiaux-pour-plasturgie-et-metallurgie')
Master('mst-genie-energetique')
Master('mst-ingenierie-environnementale-changement-climatique-et-developpement-durable')
Master('mst-intelligence-artificielle-et-sciences-de-donnees')
Master('mst-mobiquite-et-big-data')
Master('mst-modelisation-mathematique-et-science-de-donnees')
Master('mst-sciences-agroalimentaires')
Master('mst-sciences-delenvironnement')
Master('mst-sciences-du-littoralapproche-pluridisciplinaire')
Master('mst-securite-it-et-big-data')
Master('mst-systemes-informatiqueset-mobiles')

Information for mst-analyse-appliquee-etingenierie-statistique saved to Formation_master.json.
Information for mst-bases-cellulaires-etmoleculaires-en- saved to Formation_master.json.
Information for mst-environnement-aquacultureet-developpement-durable saved to Formation_master.json.
Information for mst-georessources-energetiques-et-reservoirs saved to Formation_master.json.
Information for mst-genie-civil saved to Formation_master.json.
Information for mst-genie-des-materiaux-pour-plasturgie-et-metallurgie saved to Formation_master.json.
Information for mst-genie-energetique saved to Formation_master.json.
Information for mst-ingenierie-environnementale-changement-climatique-et-developpement-durable saved to Formation_master.json.
Information for mst-intelligence-artificielle-et-sciences-de-donnees saved to Formation_master.json.
Information for mst-mobiquite-et-big-data saved to Formation_master.json.
Information for mst-modelisation-mathematique-et-science-de-donnees saved to Forma




<h1 style="text-align:center; font-size:200%;">3.Formation cycle</h1>



In [79]:
def cycle(nom_formation, file=None):
    # Construct the URL based on the provided formation name
    url = f"https://fstt.ac.ma/Portail2023/{nom_formation}/"
    
    # Fetch and parse the page
    soup = fetch_page(url)
    if not soup:
        print(f"Failed to retrieve the page for {nom_formation}.")
        return

    # Prepare the output string
    output = f"\nInformation for {nom_formation}:\n"
    
    # Find the active div containing course information
    active_div = soup.find('div', class_='eael-tabs-content')
    
    # Print course information if found
    if active_div:
        paragraphs = active_div.find_all('p', align='justify')
        if paragraphs:
            for paragraph in paragraphs:
                output += paragraph.text.strip() + '\n'
    else:
        output += "No course information found.\n"
                
    # Find the competences
    competences_div = soup.find('div', id='competences-visees-et-debouches-tab')
    
    # Print competences if found
    if competences_div:
        # Option 1: Check for ul tag
        competences_list = competences_div.find('ul')
        if competences_list:
            competences_items = competences_list.find_all('li')
            if competences_items:
                output += "\nCompetences:\n"
                for competence in competences_items:
                    output += competence.text.strip() + '\n'
        
        # Option 2: Check for p tag
        else:
            competences_paragraphs = competences_div.find_all('p', align='justify')
            if competences_paragraphs:
                output += "\nCompetences:\n"
                for paragraph in competences_paragraphs:
                    output += paragraph.text.strip() + '\n'
        
    else:
        output += "Competences not found.\n"
        
    # Find the table with id 'Mpro'
    table = soup.find('table', id='Mpro')
    
    # Print semester-wise course details if found
    if table:
        semesters = table.find_all('tr')
        for semester in semesters:
            semester_name = semester.find('th').text.strip()
            courses = semester.find('td').text.strip()
            courses = courses.replace('• ', '\n- ')
            output += f"\n{semester_name}:\n"
            output += courses + '\n'
    else:
        output += "No semester-wise course details found.\n"
    
    # Find the coordinator information
    coordinator_div = soup.find('div', id='coordinateur-tab')
    
    # Print coordinator information if found
    if coordinator_div:
        coordinator_info = coordinator_div.text.strip()
        coordinator_name, contact_info = coordinator_info.split(':', 1)
        result = f"{coordinator_name.strip()} :{contact_info.strip()}"
        output += f"\n{result}\n"
    else:
        output += "Coordinator information not found.\n"
    
    # If file is provided, write to it; otherwise, print to console
    if file:
        file.write(output)
    else:
        print(output)

In [100]:
cycle('genie-electrique-et-managementindustriel')

La formation proposée dans la spécialité Génie Electrique de la FST de Tanger a pour but de fournir au futur ingénieur en Génie Electrique tous les éléments indispensables à son insertion harmonieuse dans le monde industriel. Les enseignements associent théorie et techniques, expérimentation, projets et réalisations.
Les enseignements dispensés sont répartis en quatre groupes de matières : des enseignements à caractère général, des disciplines fondamentales, un enseignement professionnel et une formation pratique par le biais de projets et de stages.

Competences:

La production, la distribution, la transformation et l'utilisation de l'énergie électrique. la conception et la mise en oeuvre d'ensembles formés par les machines électriques et les convertisseurs qui les alimentent;
L'automatique et tous les aspects qui interviennent dans la commande des systèmes industriels;Dans la partie électronique, les systèmes sont étudiés du point de vue de leur conception, réalisation et mise en oeu

In [101]:
cycle('di-genie-industriel')

L'activité de l'ingénieur industriel s'étend des premiers contacts avec les clients jusqu'au recyclage du produit qu'il a conçu. Il assumera donc la responsabilité technique, économique et écologique de son projet. L’ingénieur mécanicien intervient dans différents domaines d’activités : automobile, sidérurgie, aéronautique, machines outils, biens de consommation, agroalimentaire.
L’ingénieur mécanicien s’intègre facilement au sein d’une PME qu’au sein d’une grande entreprise. Dans ces secteurs d’activités il occupera des postes clés dans les domaines suivants: Recherche et développement, Bureaux d’Etudes, Conception et Fabrication, Services Production, Qualité, Maintenance, Services Logistiques, Responsable projet et développement, Vente, Mise en service, Exploitation, Gestion et administration, Formation.

Competences:

L'activité de l'ingénieur industriel s'étend des premiers contacts avec les clients jusqu'au recyclage du produit qu'il a conçu. Il assumera donc la responsabilité tec

In [102]:
cycle('di-geoinformation')

Le principal objectif de cette filière est de former des ingénieurs "géo-informaticiens cartographes gestionnaires de l’information spatiale" polyvalents, qui s'intéressent prioritairement à l'organisation, au traitement, à la gestion et à la production des données géographiques et environnementales au moyen de la cartographie, de la télédétection, de l'analyse spatiale quantitative et des systèmes d'informations géographiques.
Ce programme permet à l'étudiant d’acquérir les connaissances et les compétences en géo-information et les technologies de l’information requises pour la mise en place d’infrastructures informatiques de pointe au service de la donnée géo-spatiale.
Pour atteindre cet objectif, le programme s’appui sur une démarche active de prise en main des méthodes et outils de la géo-information résolument tournée vers les applications. Ils combinent des cours théoriques et pratiques incluant aussi des travaux de terrain et des travaux personnels encadrés, des études de cas de

In [103]:
cycle('di-logiciels-et-systemes-intelligents')


Competences:

Concevoir et développer des plates formes logiciels desktop, logiciel et mobile
Concevoir et mettre en place des systèmes d’aide à la décision
Administrer une infrastructure informatique
Gérer et conduire un projet informatique

Semestre 1:

- Architecture des ordinateurs & systèmes d'exploitation
- Architecture réseaux & protocoles
- Systèmes d’informations & bases de données
- Programmation Orientée Objets en C++
- Mathématiques pour l’ingénieur
- Anglais et techniques de rédaction

Semestre 2:

- Programmation Orientée Objets en Java et Programmation Mobile
- Développement web et Frameworks (php, zend, symfony
- Théorie des graphes & Applications
- Conception Orienté Objet (UML) & Qualité Logiciel
- Statistique descriptive et inférentielle
- Management & comptabilité

Semestre 3:

- Systèmes embarqués & Internet des objets
- Gestion de projet Informatique & Planification des ressources d'entreprise (open ERP)
- Administration des bases de données 
- Méthodologies de l

In [80]:
with open("FSTT_DATA.txt", "a", encoding="utf-8") as file:
    cycle('genie-electrique-et-managementindustriel', file)
    cycle('di-genie-industriel', file)
    cycle('di-geoinformation', file)
    cycle('di-logiciels-et-systemes-intelligents', file)




<h1 style="text-align:center; font-size:200%;">Save Formation Cycle as Json</h1>



In [116]:
def cycle(nom_formation, file="Formation_cycle.json"):
    # Construct the URL based on the provided formation name
    url = f"https://fstt.ac.ma/Portail2023/{nom_formation}/"
    
    # Fetch and parse the page
    soup = fetch_page(url)
    if not soup:
        print(f"Failed to retrieve the page for {nom_formation}.")
        return

    # Initialize dictionary to store information
    data = {}
    
    # Add formation name to dictionary
    data[nom_formation] = {}

    # Find the active div containing course information
    active_div = soup.find('div', class_='eael-tabs-content')
    if active_div:
        paragraphs = active_div.find_all('p', align='justify')
        if paragraphs:
            data[nom_formation]['Description'] = "\n".join(paragraph.text.strip() for paragraph in paragraphs)
    else:
        data[nom_formation]['Description'] = "No course information found."
    
    # Find the competences
    competences_div = soup.find('div', id='competences-visees-et-debouches-tab')
    if competences_div:
        competences_list = competences_div.find('ul')
        if competences_list:
            competences_items = competences_list.find_all('li')
            if competences_items:
                data[nom_formation]['Competences'] = [competence.text.strip() for competence in competences_items]
        else:
            competences_paragraphs = competences_div.find_all('p', align='justify')
            if competences_paragraphs:
                data[nom_formation]['Competences'] = [paragraph.text.strip() for paragraph in competences_paragraphs]
    else:
        data[nom_formation]['Competences'] = ["Competences not found."]
    
    # Find the table with id 'Mpro'
    table = soup.find('table', id='Mpro')
    
    # Parse semester-wise course details
    if table:
        semesters = table.find_all('tr')
        for semester in semesters:
            semester_name = semester.find('th').text.strip()
            courses = semester.find('td').text.strip()
            courses_list = [course.strip() for course in courses.split('• ') if course.strip()]
            data[nom_formation][semester_name] = courses_list
    else:
        data[nom_formation]['Semesters'] = "No semester-wise course details found."
    
    # Find the coordinator information
    coordinator_div = soup.find('div', id='coordinateur-tab')
    
    # Parse coordinator information
    if coordinator_div:
        coordinator_info = coordinator_div.text.strip()
        coordinator_name, contact_info = coordinator_info.split(':', 1)
        data[nom_formation]['Coordinnateur'] = coordinator_name.strip()
        data[nom_formation]['Coordinnateur Email'] = contact_info.strip()
    else:
        data[nom_formation]['Coordinnateur'] = "Coordinator information not found."
        data[nom_formation]['Coordinnateur Email'] = ""

    # Read existing data from JSON file
    try:
        with open(file, 'r', encoding='utf-8') as json_file:
            existing_data = json.load(json_file)
    except FileNotFoundError:
        existing_data = {}

    # Update existing data with new data
    existing_data.update(data)
    
    # Write updated data to JSON file
    with open(file, 'w', encoding='utf-8') as json_file:
        json.dump(existing_data, json_file, indent=4, ensure_ascii=False)
    
    print(f"Information for {nom_formation} saved to {file}.")


In [117]:
cycle('genie-electrique-et-managementindustriel')
cycle('di-genie-industriel')
cycle('di-geoinformation')
cycle('di-logiciels-et-systemes-intelligents')

Information for genie-electrique-et-managementindustriel saved to Formation_cycle.json.
Information for di-genie-industriel saved to Formation_cycle.json.
Information for di-geoinformation saved to Formation_cycle.json.
Information for di-logiciels-et-systemes-intelligents saved to Formation_cycle.json.
